In [1]:
import pandas as pd

# Charger le fichier avec le bon séparateur (point-virgule) et l'encodage
df = pd.read_csv("data.csv", encoding="utf", sep=";")

# Vérification de l'affichage
print(df.head())
print(df['category'].value_counts())

   id                                            message            category
0   1  Bonjour, je veux un remboursement. Mon colis e...         réclamation
1   2            Le produit ne s’allume pas, que faire ?  problème technique
2   3  Est-ce que cette enceinte Bluetooth est compat...    question produit
3   4                  Merci pour votre service rapide !               autre
4   5  Je veux changer l’adresse de livraison de ma c...         réclamation
category
problème technique    449
réclamation           441
autre                 436
question produit      426
Name: count, dtype: int64


In [2]:
# import os.path
# import base64
# import pandas as pd
# from google.oauth2.credentials import Credentials
# from google_auth_oauthlib.flow import InstalledAppFlow
# from googleapiclient.discovery import build
# from email import message_from_bytes

# # Scope pour lecture Gmail
# SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

# def authenticate_gmail():
#     creds = None
#     if os.path.exists('token.json'):
#         creds = Credentials.from_authorized_user_file('token.json', SCOPES)
#     if not creds or not creds.valid:
#         if creds and creds.expired and creds.refresh_token:
#             creds.refresh(Request())
#         else:
#             flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
#             creds = flow.run_local_server(port=0)
#         with open('token.json', 'w') as token:
#             token.write(creds.to_json())
#     return creds

# def get_emails(service, max_results=10):
#     results = service.users().messages().list(userId='me', maxResults=max_results).execute()
#     messages = results.get('messages', [])
#     data = []

#     for msg in messages:
#         txt = service.users().messages().get(userId='me', id=msg['id'], format='full').execute()
#         headers = txt['payload']['headers']
#         subject = [h['value'] for h in headers if h['name'] == 'Subject']
#         sender = [h['value'] for h in headers if h['name'] == 'From']

#         parts = txt['payload'].get('parts', [])
#         body = ''
#         if parts:
#             for part in parts:
#                 if part['mimeType'] == 'text/plain':
#                     data_raw = base64.urlsafe_b64decode(part['body']['data']).decode('utf-8')
#                     body = data_raw
#         else:
#             body = base64.urlsafe_b64decode(txt['payload']['body']['data']).decode('utf-8')

#         data.append({
#             'from': sender[0] if sender else '',
#             'subject': subject[0] if subject else '',
#             'body': body.strip()
#         })

#     return pd.DataFrame(data)

# if __name__ == '__main__':
#     creds = authenticate_gmail()
#     service = build('gmail', 'v1', credentials=creds)
#     df = get_emails(service, max_results=10)
    
#     print(df.head())
#     df.to_csv('emails.csv', index=False)


In [3]:
import nltk
import re
import unicodedata
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('french'))

def enlever_accents(texte):
    return ''.join(c for c in unicodedata.normalize('NFD', texte) if unicodedata.category(c) != 'Mn')

def nettoyer_texte(texte):
    texte = texte.lower()
    texte = enlever_accents(texte)
    texte = re.sub(r'\d+', '', texte)               # Supprimer les chiffres
    texte = re.sub(r'[^\w\s]', '', texte)           # Supprimer ponctuation
    texte = re.sub(r'\s+', ' ', texte).strip()      # Nettoyer les espaces
    mots = texte.split()
    mots = [mot for mot in mots if mot not in stop_words]
    return " ".join(mots)

df['message_clean'] = df['message'].apply(nettoyer_texte)
print(df[['message', 'message_clean']].head(-1))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                                message  \
0     Bonjour, je veux un remboursement. Mon colis e...   
1               Le produit ne s’allume pas, que faire ?   
2     Est-ce que cette enceinte Bluetooth est compat...   
3                     Merci pour votre service rapide !   
4     Je veux changer l’adresse de livraison de ma c...   
...                                                 ...   
1746  Votre livreur a sonné à 7h30 un dimanche matin...   
1747  L'écran tactile de ma table de cuisson à induc...   
1748  Merci pour votre service de reprise de mon anc...   
1749  Ce véhicule électrique peut-il se recharger su...   
1750  Ma garantie étendue de 3 ans expire dans 2 moi...   

                                          message_clean  
0         bonjour veux remboursement colis arrive casse  
1                                 produit sallume faire  
2      estce cette enceinte bluetooth compatible iphone  
3                                  merci service rapide  
4

In [4]:
print(df.columns)


Index(['id', 'message', 'category', 'message_clean'], dtype='object')


In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['category_encoded'] = le.fit_transform(df['category'])


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['message_clean'], df['category_encoded'], test_size=0.2, random_state=42)


In [7]:
!pip install tensorflow

In [8]:
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# # Créer le tokenizer (limite vocabulaire à 10 000 mots)
# tokenizer = Tokenizer(num_words=10000)  #si par exemple [1,10,50,33] il devient [1,10,50,33,0,0,0,....,0]
# tokenizer.fit_on_texts(X_train)

# # Convertir texte en séquences d'entiers
# X_train_seq = tokenizer.texts_to_sequences(X_train)
# X_test_seq = tokenizer.texts_to_sequences(X_test)

# # Padding pour que toutes les séquences aient la même longueur (ex: max 100 mots)
# max_len = 100
# X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
# X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')


In [9]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding, LSTM, Dense

# model = Sequential()

# # Couche d'embedding : transforme chaque mot (indice) en vecteur dense de dimension 128
# model.add(Embedding(input_dim=10000, output_dim=128, input_length=max_len))

# # Couche LSTM avec 64 unités (neurones)
# model.add(LSTM(64)) #64 neuron dans sla couche

# # Couche Dense finale avec activation softmax pour classification multi-classes
# model.add(Dense(len(le.classes_), activation='softmax')) # une vecteur de 1 ligne et 4 colonne avec probabilite de message soit dans chaque category


In [10]:
# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [11]:
# model.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_split=0.1)


In [12]:
# loss, accuracy = model.evaluate(X_test_pad, y_test)
# print(f"Test accuracy: {accuracy:.2f}")


In [13]:
!pip install sentencepiece


In [14]:
from transformers import CamembertTokenizer
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")


In [15]:
def tokenize_texts(texts, max_len=30):
    return tokenizer(
        list(texts),
        truncation=True,
        padding="max_length",
        max_length=max_len,
        return_tensors="np"  # ⚠️ important pour Keras
    )

train_encodings = tokenize_texts(X_train)
test_encodings = tokenize_texts(X_test)
print("input_ids shape:", train_encodings["input_ids"].shape)        # Doit être (num_samples, max_len)
print("attention_mask shape:", train_encodings["attention_mask"].shape)


input_ids shape: (1401, 30)
attention_mask shape: (1401, 30)


In [16]:

from tensorflow.keras.utils import to_categorical

num_classes = len(le.classes_)  # ex : 4
y_train_cat = to_categorical(y_train, num_classes=num_classes)
y_test_cat = to_categorical(y_test, num_classes=num_classes)


In [17]:
!pip install tf-keras


In [18]:
from transformers import TFCamembertModel
import tensorflow as tf
from keras.saving import register_keras_serializable

# Charger CamemBERT
camembert = TFCamembertModel.from_pretrained("camembert-base")

# Entrées du modèle
input_ids = tf.keras.Input(shape=(30,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.Input(shape=(30,), dtype=tf.int32, name="attention_mask")

# 🔐 Rendre la fonction sérialisable
@register_keras_serializable()
def extract_cls(inputs):
    input_ids, attention_mask = inputs
    outputs = camembert(input_ids=input_ids, attention_mask=attention_mask)
    cls_token = outputs.last_hidden_state[:, 0, :]  # (batch_size, hidden_size)
    return cls_token

# Appliquer la couche Lambda avec la fonction sérialisable
bert_output = tf.keras.layers.Lambda(
    extract_cls,
    output_shape=(768,),
    name="extract_cls"
)([input_ids, attention_mask])

# Reste du modèle
x = tf.keras.layers.Dense(64, activation="relu")(bert_output)
output = tf.keras.layers.Dense(num_classes, activation="softmax")(x)

model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFCamembertModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing TFCamembertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFCamembertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFCamembertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertModel for predictions without further training.


In [19]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [20]:
history = model.fit(
    x={
        "input_ids": train_encodings["input_ids"],
        "attention_mask": train_encodings["attention_mask"]
    },
    y=y_train_cat,
    validation_split=0.1,
    
    batch_size=16,
    epochs=10
)

Epoch 1/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 174s 2s/step - accuracy: 0.2414 - loss: 1.4108 - val_accuracy: 0.2979 - val_loss: 1.3819
Epoch 2/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 143s 2s/step - accuracy: 0.3321 - loss: 1.3692 - val_accuracy: 0.3546 - val_loss: 1.3605
Epoch 3/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 140s 2s/step - accuracy: 0.4028 - loss: 1.3523 - val_accuracy: 0.4326 - val_loss: 1.3421
Epoch 4/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 139s 2s/step - accuracy: 0.5206 - loss: 1.3338 - val_accuracy: 0.5319 - val_loss: 1.3256
Epoch 5/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 139s 2s/step - accuracy: 0.5666 - loss: 1.3163 - val_accuracy: 0.5390 - val_loss: 1.3093
Epoch 6/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - accuracy: 0.5960 - loss: 1.2986 - val_accuracy: 0.5532 - val_loss: 1.2939
Epoch 7/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 143s 2s/step - accuracy: 0.6259 - loss: 1.2788 - val_accuracy: 0.5957 - val_loss: 1.2787
Epoch 8/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - accuracy: 0.6782 - loss: 1.2617 - val_accuracy: 0.5887 - v

In [21]:
loss, accuracy = model.evaluate(
    x={
        "input_ids": test_encodings["input_ids"],
        "attention_mask": test_encodings["attention_mask"]
    },
    y=y_test_cat
)

print(f"Accuracy: {accuracy:.4f}")


11/11 ━━━━━━━━━━━━━━━━━━━━ 37s 3s/step - accuracy: 0.6913 - loss: 1.2379
Accuracy: 0.6952


In [22]:
model.save("email_classifier_model.keras")


In [23]:
import pickle

with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)


In [24]:
tokenizer.save_pretrained("./camembert_tokenizer")


('./camembert_tokenizer\\tokenizer_config.json',
 './camembert_tokenizer\\special_tokens_map.json',
 './camembert_tokenizer\\sentencepiece.bpe.model',
 './camembert_tokenizer\\added_tokens.json')

In [25]:
import numpy as np
def predire_categorie(texte):
    # Nettoyer le texte
    texte_clean = nettoyer_texte(texte)
    
    # Tokeniser
    encoded = tokenizer(
        texte_clean,
        truncation=True,
        padding="max_length",
        max_length=30,
        return_tensors="np"
    )
    
    # Prédire
    prediction = model.predict({
        "input_ids": encoded["input_ids"],
        "attention_mask": encoded["attention_mask"]
    })
    
    # Retourner la catégorie
    predicted_class = np.argmax(prediction)
    confidence = np.max(prediction)
    
    return le.inverse_transform([predicted_class])[0], confidence

# Test
resultat, confiance = predire_categorie("")
print(f"Catégorie: {resultat}, Confiance: {confiance:.2f}") 

1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
Catégorie: autre, Confiance: 0.31


In [56]:
print(tf.__version__)

2.19.0


In [58]:
!pip freeze > requirements.txt